In [1]:
import sqlite3
import pandas as pd
import os

os.chdir("..")
# Load data from SQLite
conn = sqlite3.connect("customer_data.db")

customers = pd.read_sql("SELECT * FROM customers", conn)
subscriptions = pd.read_sql("SELECT * FROM subscriptions", conn)
usage = pd.read_sql("SELECT * FROM usage", conn)
support = pd.read_sql("SELECT * FROM support_tickets", conn)
churn = pd.read_sql("SELECT * FROM churn", conn)
payments = pd.read_sql("SELECT * FROM payments", conn)

conn.close()

# Merge into an analysis dataset
df = customers.merge(subscriptions, on="customer_id") \
              .merge(usage, on="customer_id") \
              .merge(support, on="customer_id") \
              .merge(churn, on="customer_id") \
              .merge(payments, on="customer_id")

# Data validation
df.info()
df.describe()
df.isnull().sum()

# Data cleaning
df = df[df["usage_hours"] >= 0]
df["ticket_count"] = df["ticket_count"].clip(lower=0)
df["churned_flag"] = df["churned"].map({"Yes": 1, "No": 0})

df["signup_date"] = pd.to_datetime(df["signup_date"])
df["tenure_days"] = (pd.Timestamp.today() - df["signup_date"]).dt.days
df["usage_level"] = pd.cut(
    df["usage_hours"],
    bins=[-1, 5, 20, 100],
    labels=["Low", "Medium", "High"]
)

df["support_intensity"] = pd.cut(
    df["ticket_count"],
    bins=[-1, 0, 3, 100],
    labels=["None", "Moderate", "High"]
)

df["revenue_tier"] = pd.cut(
    df["monthly_revenue"],
    bins=[0, 30, 60, 100],
    labels=["Low", "Mid", "High"]
)

# Save to csv
df.to_csv("data/cleaned/customer_analysis_dataset.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      5000 non-null   int64  
 1   signup_date      5000 non-null   object 
 2   country          5000 non-null   object 
 3   plan_type        5000 non-null   object 
 4   monthly_price    5000 non-null   int64  
 5   usage_hours      5000 non-null   float64
 6   ticket_count     5000 non-null   int64  
 7   churned          5000 non-null   object 
 8   monthly_revenue  5000 non-null   int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 351.7+ KB
